## Imports

In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
from tensorflow.python.keras.callbacks import EarlyStopping

In [2]:
df = pd.read_csv("../data/kaggleDataset/chessData.csv", nrows = 1_000_000)  # Read 5x bigger portion

# training.py

Contains code from relevant file

In [3]:
# Define the neural network architecture
model = tf.keras.Sequential([
    tf.keras.layers.Reshape((8, 8, 13), input_shape=(832,)),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
#    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
#    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='linear')
])

# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)  # Try different learning rates
model.compile(optimizer=optimizer, loss='MeanAbsoluteError')

### Fitting Model

Need to rerun the processedDataset, such that the evaluations are correct.

In [4]:
import utils

df = utils.processDF(df)

X = df.iloc[:, :-1].to_numpy()      # Select all columns except the last one (the features)
y = df.iloc[:, -1].to_numpy()       # Select the last column (the target)

c:\Users\Matthew Saad\OneDrive\College\Spring 2023\AI & ML\Final-Project-AI-ML\final\utils.py:93: FutureWarning: In a future version, passing float-dtype values and an integer dtype to DataFrame will retain floating dtype if they cannot be cast losslessly (matching Series behavior). To retain the old behavior, use DataFrame(data).astype(dtype)
  df = pd.DataFrame(combined_data, columns=column_names, dtype=np.int8)


In [5]:
df

,0,1,2,3,4,5,6,7,8,9,...,823,824,825,826,827,828,829,830,831,Evaluation
0,1,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,-10
1,1,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,56
2,1,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,-9
3,1,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,52
4,1,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,-26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,36
999996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,87
999997,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,25
999998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,105


In [6]:
df.to_csv("../data/createdData/testingSpeed1Mil.csv")

In [44]:
pd.read_csv("dfcsv")

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,823,824,825,826,827,828,829,830,831,Evaluation
0,0,1,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,-10
1,1,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,56
2,2,1,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,-9
3,3,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,52
4,4,1,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,-26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,999995,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,36
999996,999996,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,87
999997,999997,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,25
999998,999998,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,105


In [46]:
numpyDF = df.asnumpy()
np.save("dfnumpy", numpyDF)

In [47]:
numpyVersion = np.load("dfnumpy.npy")
round(numpyVersion.__sizeof__() / 1024 / 1024,2)

numpyVersion

800MB. Still big no doubt. Takes virtually no time to load in though. 

800MB * 13 = 10.4GB. Full thing in memory back on table? Need to test - does it increase exponentially, doubtful, but maybe?

In [11]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history = model.fit(X, y, epochs=50, batch_size=256, validation_split=0.2, callbacks=[early_stopping])

Epoch 1/50
313/313 [==============================] - 26s 83ms/step - loss: 54.1103 - val_loss: 54.2421
Epoch 2/50
313/313 [==============================] - 24s 78ms/step - loss: 53.6184 - val_loss: 54.3901
Epoch 3/50
313/313 [==============================] - 24s 78ms/step - loss: 53.5101 - val_loss: 54.0665
Epoch 4/50
313/313 [==============================] - 24s 77ms/step - loss: 53.3461 - val_loss: 54.0061
Epoch 5/50
313/313 [==============================] - 24s 77ms/step - loss: 53.3072 - val_loss: 54.1873
Epoch 6/50
313/313 [==============================] - 25s 79ms/step - loss: 53.1716 - val_loss: 54.0820
Epoch 7/50
313/313 [==============================] - 26s 82ms/step - loss: 53.1060 - val_loss: 54.1047
Epoch 8/50
313/313 [==============================] - 25s 80ms/step - loss: 52.9577 - val_loss: 54.0598
Epoch 9/50
313/313 [==============================] - 24s 77ms/step - loss: 52.9030 - val_loss: 54.2957
